In [4]:
from nuscenes.can_bus.can_bus_api import NuScenesCanBus
nusc_can = NuScenesCanBus(dataroot='/media/cuhp/SSD/Dataset/nuscenes/')

The CAN bus is a vehicle bus over which information such as position, velocity, acceleration, steering, lights, battery and many more are submitted. We recommend you start by reading the [README](https://github.com/nutonomy/nuscenes-devkit/tree/master/python-sdk/nuscenes/can_bus/README.md)
In BEVFormer, we only use the `pose` fields.

In [5]:
nusc_can

In [6]:
scene_name = 'scene-0001'
pose_list = nusc_can.get_messages(scene_name, 'pose')

In [7]:
len(pose_list)

964

Each value of `pose_list` contains: 
- `orientation`: a Quaternion representation of orientation
- `pos`: a global postion of ego-car
- `vel`: the velocity of ego-car
- `rotation_rate`: rotation rate

In [8]:
pose_list[1]

{'accel': [0.045534136858496645, 0.9762453196834525, 9.501466301242564],
 'orientation': [0.7460352326028626, 0.0, 0.0, 0.6659064736997176],
 'pos': [1010.1551080994742, 610.9746038474738, 0.0],
 'rotation_rate': [0.03669457882642746,
  -0.0034940228797495365,
  0.28763192892074585],
 'utime': 1531883530487320,
 'vel': [4.168599699313042, 0.0, 0.0]}

In [7]:
pose_list[0] # one example

{'accel': [0.05252802768009661, 0.9291747528573647, 9.513756543139106],
 'orientation': [0.7479305678167669, 0.0, 0.0, 0.6637769698666026],
 'pos': [1010.1436201720262, 610.8882352282457, 0.0],
 'rotation_rate': [0.040320225059986115,
  -0.002563952235504985,
  0.28492140769958496],
 'utime': 1531883530467511,
 'vel': [4.1688763951334185, 0.0, 0.0]}

In [9]:
pose_list[0].keys()

dict_keys(['accel', 'orientation', 'pos', 'rotation_rate', 'utime', 'vel'])

In [15]:
import numpy as np

In [data_converter](https://github.com/zhiqi-li/BEVFormer/blob/master/tools/data_converter/nuscenes_converter.py), we use the following function to obatain the can bus information for each sample.

In [16]:
def _get_can_bus_info(nusc, nusc_can_bus, sample):
    scene_name = nusc.get('scene', sample['scene_token'])['name']
    sample_timestamp = sample['timestamp']
    try:
        pose_list = nusc_can_bus.get_messages(scene_name, 'pose')
    except:
        return np.zeros(18)  # serveral scenes do not have can bus information.
    can_bus = []
    # during each scene, the first timestamp of can_bus may be large than the first sample's timestamp
    last_pose = pose_list[0]
    for i, pose in enumerate(pose_list):
        if pose['utime'] > sample_timestamp:
            break
        last_pose = pose # we obtain the can_bus information which is recorded before the sample recorded.
        
    _ = last_pose.pop('utime')  # useless
    pos = last_pose.pop('pos') 
    rotation = last_pose.pop('orientation')
    
    # one can_bus record contains 18 numbers
    can_bus.extend(pos) # [0:3] is the position
    can_bus.extend(rotation) # [3:7] is the orientation
    
    for key in last_pose.keys():
        can_bus.extend(pose[key])  # accel: [7, 10], rotation_rate: [10: 13], velocity: [13: 16]
    
    # the last two numbers are reserved for later calculation of rotation angle.
    can_bus.extend([0., 0.])
    
    
    return np.array(can_bus)

In [dataset](https://github.com/zhiqi-li/BEVFormer/blob/master/projects/mmdet3d_plugin/datasets/nuscenes_dataset.py#L174), we reorganize the can_bus.

In [17]:
        # actually, the nuScenes provides the rotation and translation of each sample, which is more accurate than we obtained from can bus.
        rotation = Quaternion(input_dict['ego2global_rotation'])
        translation = input_dict['ego2global_translation']

        can_bus = input_dict['can_bus']
        can_bus[:3] = translation # We use the provided translation and rotation to repalce the original translation and rotation in can bus
        can_bus[3:7] = rotation

        patch_angle = quaternion_yaw(rotation) / np.pi * 180 # we get the yaw angle of ego car
        can_bus[-2] = patch_angle / 180 * np.pi # this angle is kept unchanged.
        can_bus[-1] = patch_angle # this angle is used to compute the detal of adjacent timestamps.

NameError: name 'Quaternion' is not defined

In [dataset](https://github.com/zhiqi-li/BEVFormer/blob/master/projects/mmdet3d_plugin/datasets/nuscenes_dataset.py#L93), we compute the delta orientation and position of adjacent timestamps

In [ ]:
        prev_pos = None
        prev_angle = None
        for i, each in enumerate(queue):
            metas_map[i] = each['img_metas'].data
            if i == 0:
                metas_map[i]['prev_bev'] = False
                prev_pos = copy.deepcopy(metas_map[i]['can_bus'][:3])
                prev_angle = copy.deepcopy(metas_map[i]['can_bus'][-1])
                metas_map[i]['can_bus'][:3] = 0
                metas_map[i]['can_bus'][-1] = 0
            else:
                metas_map[i]['prev_bev'] = True
                tmp_pos = copy.deepcopy(metas_map[i]['can_bus'][:3])
                tmp_angle = copy.deepcopy(metas_map[i]['can_bus'][-1])
                metas_map[i]['can_bus'][:3] -= prev_pos
                metas_map[i]['can_bus'][-1] -= prev_angle
                prev_pos = copy.deepcopy(tmp_pos)
                prev_angle = copy.deepcopy(tmp_angle)